## Topic Modeling with Gensim

pip install gensim
pip install pyldavis

In [20]:
from sklearn.datasets import fetch_20newsgroups#뉴스기사 데이터 다운

categories = ['alt.atheism', 'talk.religion.misc', 'comp.graphics', 'sci.space']

newsgroups_train = fetch_20newsgroups(subset='train',
#메일 내용에서 hint가 되는 부분을 삭제 - 순수하게 내용만으로
                                      remove=('headers', 'footers', 'quotes'),
                                      categories=categories)

In [21]:
# 필요한 library들을 import
from nltk.corpus import stopwords
cachedStopWords = stopwords.words("english")

from nltk import word_tokenize
from nltk.stem.porter import PorterStemmer
import re

In [22]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\gksquf\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [23]:
def tokenize(text):
    min_length = 3 #최소 단어 크기
    # nltk의 tokenizer를 이용해서 word 추출한 후에 소문자로 변환
    words = map(lambda word: word.lower(), word_tokenize(text))
    #words = word_tokenize(text.lower()) #이렇게 해도 되는지 확인
    # stopwords 제외
    words = [word for word in words if word not in cachedStopWords]
    # portr stemmer 적용
    tokens = (list(map(lambda token: PorterStemmer().stem(token),words)))
    #tokens = [PorterStemmer().stem(token) for token in words]  #이렇게 해도 되는지 확인
    #알파벳으로 이루어진 단어들만 추출
    p = re.compile('[a-zA-Z]+');
    filtered_tokens = list(filter (lambda token: p.match(token) and len(token) >= min_length,tokens))
    #filtered_tokens = [token for token in tokens if p.match(token) and len(token) >= min_length]   #이렇게 해도 되는지 확인
    return filtered_tokens

In [24]:
docs = [tokenize(doc) for doc in newsgroups_train.data]#토큰화

In [25]:
print(newsgroups_train.data[0])#전처리 된 데이터
print(docs[0])

Hi,

I've noticed that if you only save a model (with all your mapping planes
positioned carefully) to a .3DS file that when you reload it after restarting
3DS, they are given a default position and orientation.  But if you save
to a .PRJ file their positions/orientation are preserved.  Does anyone
know why this information is not stored in the .3DS file?  Nothing is
explicitly said in the manual about saving texture rules in the .PRJ file. 
I'd like to be able to read the texture rule information, does anyone have 
the format for the .PRJ file?

Is the .CEL file format available from somewhere?

Rych
['notic', 'save', 'model', 'map', 'plane', 'posit', 'care', 'file', 'reload', 'restart', 'given', 'default', 'posit', 'orient', 'save', 'file', 'positions/orient', 'preserv', 'anyon', 'know', 'inform', 'store', 'file', 'noth', 'explicitli', 'said', 'manual', 'save', 'textur', 'rule', 'file', 'like', 'abl', 'read', 'textur', 'rule', 'inform', 'anyon', 'format', 'file', 'file', 'format', 'a

Gensim으로 topic modeling을 하기 위해서는 Dictionary, Corpus, Model의 3단계를 거쳐야 함

In [26]:
#pip install gensim

In [27]:
from gensim.corpora import Dictionary#딕셔너리로 전처리

# Create a dictionary representation of the documents.
dictionary = Dictionary(docs)
print('Number of unique words in initital documents:', len(dictionary))

# Filter out words that occur less than 10 documents, or more than 20% of the documents.
dictionary.filter_extremes(no_below=3, no_above=0.5)
print('Number of unique words after removing rare and common words:', len(dictionary))

Number of unique words in initital documents: 19661
Number of unique words after removing rare and common words: 5628


In [28]:
corpus = [dictionary.doc2bow(doc) for doc in docs]
print('Number of unique tokens: %d' % len(dictionary))
print('Number of documents: %d' % len(corpus))#코퍼스와 딕셔너리 확인

Number of unique tokens: 5628
Number of documents: 2034


In [29]:
from gensim.models import LdaModel
# Set training parameters.
num_topics = 25
chunksize = 500 # size of the doc looked at every pass
passes = 20 # number of passes through documents
iterations = 40
eval_every = 1  # Don't evaluate model perplexity, takes too much time.

# Make a index to word dictionary.
temp = dictionary[0]  # This is only to "load" the dictionary.
id2word = dictionary.id2token

%time model = LdaModel(corpus=corpus, id2word=id2word, chunksize=chunksize, \
                       alpha='auto', eta='auto', \
                       iterations=iterations, num_topics=num_topics, \
                       passes=passes, eval_every=eval_every)

Wall time: 1min 10s


In [30]:
#! pip install pyldavis 

import pyLDAvis
import pyLDAvis.gensim_models as gensimvis#시각화 하기/코드변경(이름이 바뀌었다고 함)
pyLDAvis.enable_notebook()

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

In [31]:
#!pip install pyLDAvis #시각화
import pyLDAvis
import pyLDAvis.gensim_models

pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(model, corpus, dictionary)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1     -0.208004 -0.029276       1        1  24.437149
4     -0.004741  0.179885       2        1  13.557010
22    -0.149346  0.168751       3        1   7.661653
15    -0.092506  0.173105       4        1   6.149873
23    -0.112687 -0.005833       5        1   6.132999
24    -0.172468  0.163043       6        1   5.439284
5     -0.080987  0.042561       7        1   3.818179
0     -0.082409 -0.097741       8        1   3.299424
7     -0.103879  0.004217       9        1   3.130708
18    -0.093329 -0.179775      10        1   3.088324
9     -0.075264 -0.171866      11        1   2.830010
12    -0.146064  0.064483      12        1   2.802065
19     0.055354 -0.212371      13        1   2.794583
2     -0.033021 -0.243365      14        1   2.404464
3     -0.011872 -0.108201      15        1   2.389362
10    -0.093743  0.028200      16        1   1.907691
13     0.141971  0.104293      17        1   1.654951
14     0.030245 -0.009917      18        1   1.595328
16     0.071508 -0.031898      19        1   1.409226
6      0.117306  0.022168      20        1   1.050595
20     0.194270  0.039898      21        1   0.732257
11     0.206681  0.051274      22        1   0.615630
17     0.197744 -0.005214      23        1   0.605010
21     0.226331  0.024557      24        1   0.313734
8      0.218912  0.029022      25        1   0.180492, topic_info=        Term         Freq        Total Category  logprob  loglift
60       n't  1832.000000  1832.000000  Default  30.0000  30.0000
724      god   827.000000   827.000000  Default  29.0000  29.0000
284     imag   857.000000   857.000000  Default  28.0000  28.0000
123    think   879.000000   879.000000  Default  27.0000  27.0000
130    would  1247.000000  1247.000000  Default  26.0000  26.0000
...      ...          ...          ...      ...      ...      ...
131     aiaa     0.754677    10.560895  Topic25  -6.0555   3.6786
1226   innoc     2.811071    41.649532  Topic25  -4.7405   3.6215
400   accept     3.107421   168.112385  Topic25  -4.6402   2.3264
641   longer     1.545175    43.636775  Topic25  -5.3389   2.9765
1753    caus     1.466224   346.353318  Topic25  -5.3913   0.8525

[1442 rows x 6 columns], token_table=      Topic      Freq   Term
term                        
4244     10  0.968330    a.a
0         1  0.430397    abl
0         6  0.381673    abl
0        19  0.170535    abl
4374     15  0.937702  abort
...     ...       ...    ...
618      15  0.235923   year
618      16  0.006493   year
618      19  0.114715   year
3708     22  0.770365  yield
3606     16  0.961128   zero

[2888 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 5, 23, 16, 24, 25, 6, 1, 8, 19, 10, 13, 20, 3, 4, 11, 14, 15, 17, 7, 21, 12, 18, 22, 9])